# NOTEBOOK 3: Exploratory data analysis

## Visualization
---
### Boxplots
The module `seaborn` implements a lot of nice statistical plots (see: https://seaborn.pydata.org/examples/index.html)

Standardization is done the remove any physical units from the data and ensure a mean on zero and a standard deviation of one

## Correlation analysis
---

### Pearson correlation coefficient
The Pearson correlation coefficient is a measure of linear correlation between two variables `X` and `y`. It quantifies how well the relationship between two variables can be described by a straight line

The null hypothesis (H0): There is no linear correlation between `X` and `y` (population correlation $\rho=0$)

#### Heat maps

## Dimension reduction
---
### PCA
Principal Component Analysis is a dimensionality reduction technique. It transforms a dataset with possibly correlated variables into a set of linearly uncorrelated variables called principal components.